In [31]:
import pandas as pd
import numpy as np

In [33]:
test = pd.read_csv("../../../Data/RCT_data/rct_test.csv" )
test_result = pd.read_csv("RCT_Chiresult" )
test[['pred1','pred2','pred3','pred4','pred5','pred6','pred7']]=test_result[['1', '2', '3', '4', '5', '6', '7']]
test_pivot = test.copy()

# Roi for each treatment, assume delta_cost = [0.3,0.5,0.7,1.0,1.3,1.6,1.9], base_cost = 0.1
test_pivot['t_1_roi'] = test_pivot['pred1']/0.3
test_pivot['t_2_roi'] = test_pivot['pred2']/0.5
test_pivot['t_3_roi'] = test_pivot['pred3']/0.7
test_pivot['t_4_roi'] = test_pivot['pred4']/1.0
test_pivot['t_5_roi'] = test_pivot['pred5']/1.3
test_pivot['t_6_roi'] = test_pivot['pred6']/1.6
test_pivot['t_7_roi'] = test_pivot['pred7']/1.9

# For each user i, select the treatment with max ROI
test_pivot['best_treatment_roi'] = np.where((test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']] < 0).all(axis=1),0,test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']].max(axis=1))
test_pivot['best_treatment']=np.where((test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']] <0).all(axis=1),'control',test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']].idxmax(axis=1))


# Sort all users according to each user's max ROI in a descending order.
test_pivot = test_pivot.sort_values(by=['best_treatment_roi'],ascending=False)

# The cost for all users when selecting the max ROI treatment
coin_map = {'control':0,'t_1_roi': 0.3,'t_2_roi':0.5,'t_3_roi':0.7,'t_4_roi':1,'t_5_roi':1.3,'t_6_roi':1.6,'t_7_roi':1.9}
test_pivot['delta_coin'] = test_pivot['best_treatment'].map(coin_map)
test_pivot['coin'] = test_pivot['delta_coin'] + 0.1
test_pivot['cum_expense'] = (test_pivot['coin']).cumsum()
test_pivot['delta_cum_expense'] = (test_pivot['delta_coin']).cumsum()
test_pivot['base_expense'] = 0.1
test_pivot['cum_base_expense'] = test_pivot['base_expense'].cumsum()

min_cost = test_pivot.base_expense.sum()

max_cost = test_pivot.coin.sum()


for budget in [300,400,500,600,700]:

    test_pivot['extra_expense'] = budget  -  min_cost
    test_pivot['cut_off_point'] = test_pivot['delta_cum_expense'] <= test_pivot['extra_expense']
    test_pivot['final_coin'] =  np.where(test_pivot['cut_off_point'],test_pivot['coin'],0.1)

    coin_map = {0:0.1,1: 0.4,2:0.6,3:0.8,4:1.1,5:1.4,6:1.7,7:2.0}
    test_pivot['treatment'] = test_pivot['exp_group'].map(coin_map)

    test_pivot['final_coin'] = test_pivot['final_coin'].round(1)
    test_pivot['treatment'] = test_pivot['treatment'].round(1)

    test_pivot['if_same'] = test_pivot['treatment'] == test_pivot['final_coin']
    test_pivot['tmp_label'] =  np.where(test_pivot['if_same'],test_pivot['treatment'],0)
    
    test_pivot.to_csv(str(budget)+'RCT_Chi_final.csv')

In [34]:
test = pd.read_csv("../../../Data/RCT_data/rct_test.csv" )
test_result = pd.read_csv("RCT_EDresult" )
test[['pred1','pred2','pred3','pred4','pred5','pred6','pred7']]=test_result[['1', '2', '3', '4', '5', '6', '7']]
test_pivot = test.copy()

# Roi for each treatment, assume delta_cost = [0.3,0.5,0.7,1.0,1.3,1.6,1.9], base_cost = 0.1
test_pivot['t_1_roi'] = test_pivot['pred1']/0.3
test_pivot['t_2_roi'] = test_pivot['pred2']/0.5
test_pivot['t_3_roi'] = test_pivot['pred3']/0.7
test_pivot['t_4_roi'] = test_pivot['pred4']/1.0
test_pivot['t_5_roi'] = test_pivot['pred5']/1.3
test_pivot['t_6_roi'] = test_pivot['pred6']/1.6
test_pivot['t_7_roi'] = test_pivot['pred7']/1.9

# For each user i, select the treatment with max ROI
test_pivot['best_treatment_roi'] = np.where((test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']] < 0).all(axis=1),0,test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']].max(axis=1))
test_pivot['best_treatment']=np.where((test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']] <0).all(axis=1),'control',test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']].idxmax(axis=1))


# Sort all users according to each user's max ROI in a descending order.
test_pivot = test_pivot.sort_values(by=['best_treatment_roi'],ascending=False)

# The cost for all users when selecting the max ROI treatment
coin_map = {'control':0,'t_1_roi': 0.3,'t_2_roi':0.5,'t_3_roi':0.7,'t_4_roi':1,'t_5_roi':1.3,'t_6_roi':1.6,'t_7_roi':1.9}
test_pivot['delta_coin'] = test_pivot['best_treatment'].map(coin_map)
test_pivot['coin'] = test_pivot['delta_coin'] + 0.1
test_pivot['cum_expense'] = (test_pivot['coin']).cumsum()
test_pivot['delta_cum_expense'] = (test_pivot['delta_coin']).cumsum()
test_pivot['base_expense'] = 0.1
test_pivot['cum_base_expense'] = test_pivot['base_expense'].cumsum()

min_cost = test_pivot.base_expense.sum()

max_cost = test_pivot.coin.sum()


for budget in [300,400,500,600,700]:

    test_pivot['extra_expense'] = budget  -  min_cost
    test_pivot['cut_off_point'] = test_pivot['delta_cum_expense'] <= test_pivot['extra_expense']
    test_pivot['final_coin'] =  np.where(test_pivot['cut_off_point'],test_pivot['coin'],0.1)

    coin_map = {0:0.1,1: 0.4,2:0.6,3:0.8,4:1.1,5:1.4,6:1.7,7:2.0}
    test_pivot['treatment'] = test_pivot['exp_group'].map(coin_map)

    test_pivot['final_coin'] = test_pivot['final_coin'].round(1)
    test_pivot['treatment'] = test_pivot['treatment'].round(1)

    test_pivot['if_same'] = test_pivot['treatment'] == test_pivot['final_coin']
    test_pivot['tmp_label'] =  np.where(test_pivot['if_same'],test_pivot['treatment'],0)
    
    test_pivot.to_csv(str(budget)+'RCT_ED_final.csv')

In [35]:
test = pd.read_csv("../../../Data/RCT_data/rct_test.csv" )
test_result = pd.read_csv("RCT_CTSresult" )
test[['pred1','pred2','pred3','pred4','pred5','pred6','pred7']]=test_result[['1', '2', '3', '4', '5', '6', '7']]
test_pivot = test.copy()

# Roi for each treatment, assume delta_cost = [0.3,0.5,0.7,1.0,1.3,1.6,1.9], base_cost = 0.1
test_pivot['t_1_roi'] = test_pivot['pred1']/0.3
test_pivot['t_2_roi'] = test_pivot['pred2']/0.5
test_pivot['t_3_roi'] = test_pivot['pred3']/0.7
test_pivot['t_4_roi'] = test_pivot['pred4']/1.0
test_pivot['t_5_roi'] = test_pivot['pred5']/1.3
test_pivot['t_6_roi'] = test_pivot['pred6']/1.6
test_pivot['t_7_roi'] = test_pivot['pred7']/1.9

# For each user i, select the treatment with max ROI
test_pivot['best_treatment_roi'] = np.where((test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']] < 0).all(axis=1),0,test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']].max(axis=1))
test_pivot['best_treatment']=np.where((test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']] <0).all(axis=1),'control',test_pivot[['t_1_roi','t_2_roi','t_3_roi','t_4_roi','t_5_roi','t_6_roi','t_7_roi']].idxmax(axis=1))


# Sort all users according to each user's max ROI in a descending order.
test_pivot = test_pivot.sort_values(by=['best_treatment_roi'],ascending=False)

# The cost for all users when selecting the max ROI treatment
coin_map = {'control':0,'t_1_roi': 0.3,'t_2_roi':0.5,'t_3_roi':0.7,'t_4_roi':1,'t_5_roi':1.3,'t_6_roi':1.6,'t_7_roi':1.9}
test_pivot['delta_coin'] = test_pivot['best_treatment'].map(coin_map)
test_pivot['coin'] = test_pivot['delta_coin'] + 0.1
test_pivot['cum_expense'] = (test_pivot['coin']).cumsum()
test_pivot['delta_cum_expense'] = (test_pivot['delta_coin']).cumsum()
test_pivot['base_expense'] = 0.1
test_pivot['cum_base_expense'] = test_pivot['base_expense'].cumsum()

min_cost = test_pivot.base_expense.sum()

max_cost = test_pivot.coin.sum()


for budget in [300,400,500,600,700]:

    test_pivot['extra_expense'] = budget  -  min_cost
    test_pivot['cut_off_point'] = test_pivot['delta_cum_expense'] <= test_pivot['extra_expense']
    test_pivot['final_coin'] =  np.where(test_pivot['cut_off_point'],test_pivot['coin'],0.1)

    coin_map = {0:0.1,1: 0.4,2:0.6,3:0.8,4:1.1,5:1.4,6:1.7,7:2.0}
    test_pivot['treatment'] = test_pivot['exp_group'].map(coin_map)

    test_pivot['final_coin'] = test_pivot['final_coin'].round(1)
    test_pivot['treatment'] = test_pivot['treatment'].round(1)

    test_pivot['if_same'] = test_pivot['treatment'] == test_pivot['final_coin']
    test_pivot['tmp_label'] =  np.where(test_pivot['if_same'],test_pivot['treatment'],0)
    
    test_pivot.to_csv(str(budget)+'RCT_CTS_final.csv')